In [1]:
import matplotlib.pyplot as plt
import torchvision as tv
from torchvision import transforms, utils


import torch.nn.functional as F
import torch 
import torch.nn as nn
from torch.autograd import Variable
import torchvision.models as models
# from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
# from PIL import Image
# import numpy as np
import torch.optim as optim
# import os
import time

import loadData

In [2]:
device = torch.device("cuda:2")

In [3]:
# 日志记录模块2
import logging
logger = logging.getLogger(__name__)
logger.setLevel(level = logging.INFO)
handler = logging.FileHandler("./FFT30dB-all.txt")
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

console = logging.StreamHandler()
console.setLevel(logging.INFO)

logger.addHandler(handler)
logger.addHandler(console)

In [4]:
startTime = time.time()

In [5]:
transforms = tv.transforms.Compose([
    transforms.ToTensor(),
])


numOfBatch = 9
pixel = 64
flag = 0
root = f'./{numOfBatch}/data'
batchSize = 4
train_data = loadData.MyDataset(txt=f'files_train.txt', transform=None)
test_data = loadData.MyDataset(txt=f'files_test.txt', transform=None)


train_loader = DataLoader(dataset=train_data, batch_size=batchSize, shuffle=True, num_workers=2)
test_loader = DataLoader(dataset=test_data, batch_size=batchSize, shuffle=True, num_workers=2)


print(train_loader)
print(test_loader)

In [6]:
print(len(train_data))
print(len(test_loader))

810
135


In [7]:
data = iter(train_loader)

# print(type(data))
# print(len(data))
data_, label = data.next()
print(train_loader.batch_size)
print(data_.shape)
# print(label)
# print(data_, label)

(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
4
torch.Size([4, 1, 20, 1])
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)


In [8]:
# x = torch.randn(4, 1, 1200, 2).float()
# conv = nn.Conv2d(in_channels = 1, out_channels = 128, kernel_size = (10, 1))
# x = conv(x)
# x.shape
# conv2 = nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = (3, 2))
# x = conv2(x)
# x.shape


In [9]:
# 256*1189

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (5, 1))
#         conv = nn.Conv2d(in_channels = 1, out_channels = 128, kernel_size = (10, 1))

        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (3, 1))
        
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(224, 126)
        self.fc2 = nn.Linear(126, 100)
        self.fc3 = nn.Linear(100, 54)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
    
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

In [11]:
# TensorBoard
# from tensorboardX import SummaryWriter
# net = Net()
# # writer = SummaryWriter('writer928') #建立一个保存数据用的东西
# dummy_input = torch.rand(2,160,400)  # 假设输入20张1*28*28的图片
# with SummaryWriter(comment='Net1') as w:
#     w.add_graph(net, (dummy_input,))

In [12]:
net = Net()

net = net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=net.parameters(), lr=0.0001, momentum=0.9)

epochs = 5000
average_loss_series = []

In [13]:
for a, b in enumerate(train_loader):
    inputs, labels = b
    print(inputs.shape)
    break

(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
torch.Size([4, 1, 20, 1])


In [ ]:
%matplotlib inline


if __name__ == '__main__':
    for epoch in range(epochs):
        running_loss = 0.0
        running_acc = 0.0

        for i, data in enumerate(train_loader):
#             print("i:", i )
#             print("data: ", data[0])
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

#             inputs = inputs.permute(0, 2, 1)
            
            inputs = torch.tensor(inputs, dtype=torch.float32, device = device)

            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, dim=1)
            total = labels.size(0)
#             print('原来的值:{0}, 预测的值:{1}'.format(labels, predicted))
            running_correct = (predicted == labels).sum()
            running_acc += running_correct
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            loss.backward()
            optimizer.step()

            # 每loopNum个batch打印一次训练状态
            loopNum = 20
            if i % loopNum == loopNum - 1:
                average_loss = running_loss / loopNum
                logger.info('[{0}, {1}] loss: {2}'.format(epoch + 1, i + 1, average_loss))
                logger.info('设备号为: {0}，预测的设备号为:{1}'.format(labels, predicted))
#                 logger.info(torch.cuda.current_device ())
                running_acc = running_acc.float()
                average_acc = running_acc / loopNum / total
                
                logger.info('[{0}, {1}] acc: {2}'.format(epoch + 1, i + 1, average_acc))
                average_loss_series.append(average_loss)
                running_loss = 0.0
                running_acc = 0.0
        if epoch == 78:
            torch.save(net, f'/workspace/DATASET/model_78.pkl')

    x = range(0, len(average_loss_series))
    plt.figure()
    plt.plot(x, average_loss_series)
#     plt.savefig("Loss.png")
    plt.savefig("Loss.png", bbox_inches="tight")
    plt.show()
    

    # %%
    # 在测试集上测试
    realLabel = []
    predictedLabel = []

    def correct_rate(net, testloader):
        correct = 0.0
        total = 0.0
        count = 0
        correctpredict = 0

        for data in testloader:
            images, labels = data
            images, labels = images.to(device),labels.to(device)
            realLabel.append([int(label) for label in labels])
    
            tests = torch.tensor(images, dtype=torch.float32,device=device)
            outputs = net(tests)

            _, predicted = torch.max(outputs.data, dim=1)
            logger.info('设备号为: {0}，模型训练后得到的设备号为:{1}'.format(labels, predicted))
#             deviceNumber = torch.cuda.current_device ()
#             logger.info(deviceNumber)
            
            labelsn = labels.cpu().numpy()
            pn = predicted.cpu().numpy()
            count += len(labelsn)
            
            for i in range(len(labelsn)):
                if labelsn[i] == pn[i]:
                    correctpredict += 1
                
            
            logger.info('目前一共测试了{0}组，预测成功的一共有{1}组'.format(count, correctpredict))
            predictedLabel.append([int(label) for label in predicted])
            
            total += labels.size(0)
            correct += (predicted == labels).sum()

            
        logger.info("total的值为：{total}")
#         return 100 * correct  # / total
        return 100 * correct/ total 


    correct = correct_rate(net, test_loader)
    logger.info(f'{len(test_loader) * 3}张测试集中准确率为： {correct}%')

    # %%
    import itertools
    from sklearn.metrics import confusion_matrix

    realLabel = list(itertools.chain.from_iterable(realLabel))
    predictedLabel = list(itertools.chain.from_iterable(predictedLabel))

    cm = confusion_matrix(realLabel, predictedLabel)
    logger.info(cm)

#     logger.info('the running time is:{1}'.format(time.time() - startTime))
    torch.save(net, f'/workspace/PytorchCnn/models/model_{epoch}_{pixel}_{flag}.pkl')
    

(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)


[1, 20] loss: 4.091192138195038


(1, 20, 1)


...

(1, 20, 1)


[46, 180] acc: 0.05000000074505806


(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)


[46, 200] loss: 3.459103059768677
设备号为: tensor([30, 28, 11, 52], device='cuda:2')，预测的设备号为:tensor([15,  1,  1, 45], device='cuda:2')
[46, 200] acc: 0.07500000298023224


(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)
(1, 20, 1)


In [ ]:
torch.cuda.current_device ()

In [ ]:
aa = iter(train_loader)
a, b = aa.next()
print(a[0].shape)
print(a)

In [ ]:
input = torch.randn(20, 16, 50, 100)
input.shape

In [ ]:
m2=nn.Conv2d(16,33,(3,5),stride=(2,1),padding=(4,2))
inpu1=Variable(torch.randn(20,16,50,100))
print(inpu1.shape)
output=m2(inpu1)
print(output.shape)

In [ ]:
total

In [ ]:
torch.cuda.current_device () 

In [ ]:
x = range(0, len(average_loss_series))
plt.figure()
plt.plot(x, average_loss_series)
plt.savefig("Loss1.png", bbox_inches="tight")
plt.show()